In [1]:
import jetson_inference
from jetson_utils import cudaToNumpy, videoSource, cudaDeviceSynchronize
import ipywidgets.widgets as widgets
import IPython.display
from PIL import Image
import matplotlib.pyplot as plt
import time
from io import BytesIO
import cv2
import uuid
import board
import busio


In [2]:
class Wheels:
    stop_cmd: bytes = bytes([0x0])
    forward_cmd: bytes = bytes([0x1])
    reverse_cmd: bytes = bytes([0x2])
    left_cmd: bytes = bytes([0x3])
    right_cmd: bytes = bytes([0x4])
    i2c: busio.I2C
    address = 0x15

    def __init__(self): 
        self.i2c = busio.I2C(board.SCL_1, board.SDA_1)

    def stop(self):
        self.i2c.writeto(self.address, self.stop_cmd)

    def go_forward(self):
        self.i2c.writeto(self.address, self.forward_cmd)

    def reverse(self):
        self.i2c.writeto(self.address, self.reverse_cmd)

    def go_left(self):
        self.i2c.writeto(self.address, self.left_cmd)

    def go_right(self):
        self.i2c.writeto(self.address, self.right_cmd)

In [3]:
source = "csi://0"
wheels = Wheels()
input = videoSource(source)


[gstreamer] initialized gstreamer, version 1.20.3.0
[gstreamer] gstCamera -- attempting to create device csi://0
[gstreamer] gstCamera pipeline string:
[gstreamer] nvarguscamerasrc sensor-id=0 ! video/x-raw(memory:NVMM), width=(int)1280, height=(int)720, framerate=30/1, format=(string)NV12 ! nvvidconv flip-method=2 ! video/x-raw ! appsink name=mysink
[gstreamer] gstCamera successfully created device csi://0
[video]  created gstCamera from csi://0
------------------------------------------------
gstCamera video options:
------------------------------------------------
  -- URI: csi://0
     - protocol:  csi
     - location:  0
  -- deviceType: csi
  -- ioType:     input
  -- width:      1280
  -- height:     720
  -- frameRate:  30
  -- numBuffers: 4
  -- zeroCopy:   true
  -- flipMethod: rotate-180
------------------------------------------------


In [4]:
def wait_action():
    wheels.stop()

def go_forward(change):
    wheels.go_forward()

def go_backward(change):
    wheels.reverse()

def go_left(change):
    wheels.go_left()

def go_right(change):
    wheels.go_right()

def stop(change):
    wheels.stop()

In [5]:
output_image = widgets.Image(format='png', width=1280, height=720)
display(output_image)

Image(value=b'', height='720', width='1280')

In [6]:
import threading, time

class DisplayCaptureBackground(threading.Thread):
    display_capture = True
    
    def stop(self):
        self.display_capture = False
    
    def run(self):
        while self.display_capture:
            image = input.Capture()  
            if (image is not None):
                array = cudaToNumpy(image)
                f = BytesIO()
                final_image = Image.fromarray(array)
                final_image.save(f, format="jpeg")
                output_image.value = f.getvalue()
                del image
    
display_background = DisplayCaptureBackground()
display_background.start()

[gstreamer] opening gstCamera for streaming, transitioning pipeline to GST_STATE_PLAYING
[gstreamer] gstreamer changed state from NULL to READY ==> mysink
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter1
[gstreamer] gstreamer changed state from NULL to READY ==> nvvconv0
[gstreamer] gstreamer changed state from NULL to READY ==> capsfilter0
[gstreamer] gstreamer changed state from NULL to READY ==> nvarguscamerasrc0
[gstreamer] gstreamer changed state from NULL to READY ==> pipeline0
[gstreamer] gstreamer changed state from READY to PAUSED ==> capsfilter1
[gstreamer] gstreamer changed state from READY to PAUSED ==> nvvconv0
[gstreamer] gstreamer changed state from READY to PAUSED ==> capsfilter0
[gstreamer] gstreamer stream status CREATE ==> src
[gstreamer] gstreamer changed state from READY to PAUSED ==> nvarguscamerasrc0
[gstreamer] gstreamer changed state from READY to PAUSED ==> pipeline0
[gstreamer] gstreamer message new-clock ==> pipeline0
[gstreamer] gstrea

In [13]:
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', layout=button_layout, icon='stop')
forward_button = widgets.Button(description='forward', layout=button_layout, icon='arrow-up')
backward_button = widgets.Button(description='backward', layout=button_layout, icon='arrow-down')
left_button = widgets.Button(description='left', layout=button_layout, icon='arrow-left')
right_button = widgets.Button(description='right', layout=button_layout, icon='arrow-right')

output = widgets.Output()

stop_button.on_click(stop)
left_button.on_click(go_left)
right_button.on_click(go_right)
backward_button.on_click(go_backward)
forward_button.on_click(go_forward)

center_box_layout = layout=widgets.Layout(align_self='center')
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=center_box_layout)
controls_box = widgets.VBox([forward_button, middle_box, backward_button])
display(controls_box, output)

Output()